In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import wandb
from sklearn.model_selection import train_test_split


In [2]:
total_out_num = 5
input_number = 1868
out_num = 1

In [3]:
from torch.utils.data import Dataset

class IR_wine_dataset(Dataset):
    def __init__(self, target_n, nums, data_file, label_file, transform=None, target_transform=None, sep = ","):
        self.target_n = target_n
        self.pattern_n = pd.read_csv(nums, sep=',')
        self.spec_labels = pd.read_csv(label_file, sep=',').iloc[:,[0,self.target_n]]
        self.spec = pd.read_csv(data_file, sep=',')
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(np.array(self.pattern_n))

    def __getitem__(self, idx):
        number = self.pattern_n.iloc[idx]
        sp = torch.from_numpy(1000*np.array((self.spec.loc[self.spec.iloc[:,0]==number[0]]).iloc[:,1:])).to(torch.float32)
        label = torch.from_numpy(np.array((self.spec_labels.loc[self.spec_labels.iloc[:,0].astype(int)==number[0]]).iloc[:,1:])).to(torch.float32).squeeze(0)
        number = self.pattern_n.iloc[idx]
        
        if self.transform:
            sp = self.transform(sp)
        if self.target_transform:
            label = self.target_transform(label)
        return np.array(number), sp, label

In [4]:
#Calculate min and max
def mean_std(train_dataloader):
    data = next(iter(train_dataloader))
    MEAN = torch.mean(data[1],0)
    STD = torch.std(data[1],0)
    return MEAN, STD

In [5]:
class Normalize:
    def __init__(self,feat_mean,feat_std):
        assert feat_mean.shape == feat_std.shape
        self.feat_mean = feat_mean
        self.feat_std = feat_std

    def __call__(self,x):
        return (x - self.feat_mean) / (self.feat_std)

In [6]:
#1D CNN class (basic)
class OneDCNN(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=4, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Conv1d(in_channels=4, out_channels=8, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(461*8, 32),
            nn.Sigmoid(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

In [7]:
def reset_weights(m):
    '''
    Try resetting model weights to avoid
    weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [8]:
#Write the outputs of the network
def write_predictions(parameter, N, model_name, split_path,dataloader,feat_num,dset):
    checkpoint = torch.load(split_path + 'model'+model_name+'.pth')
    N.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_best_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    N.eval()

    out, true = torch.zeros(1, feat_num+1), torch.zeros(1, feat_num+1)
    for num, specs, labels in dataloader:
        outputs = N(specs)
        outputs[outputs<0]=0
        
        out = torch.cat((out, torch.cat((num, outputs), -1)),0)
        true = torch.cat((true, torch.cat((num, labels), -1)),0)

    a = ['sample_number',parameter]
    
    pd.DataFrame(out.detach().numpy()).to_csv(split_path + 'Y_out_'+dset+'.csv',sep=',', header = a)
    pd.DataFrame(true.detach().numpy()).to_csv(split_path + 'Y_true_'+dset+'.csv',sep=',', header = a)

In [9]:
gen_path = 'G:\\wine_half_data\\'
data_file = 'G:\\wine_half_data\\data\\wines_november.csv'
label_file = 'G:\\wine_half_data\\data\\wine_concentrations.csv'

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134],[139,15],[287,403],[32,29]] #cross-validation folds

target = 'ethanol_pct'
case_name = 'wine_november_IR_CNN_100_epoch_'+target

targets_num = {'ethanol_pct': 1, 'sugar_gL': 2, 'acids_gL': 3, 'glycerol_gL': 4, 'sulfur_dioxide_gL': 5}
case_path = gen_path+case_name+'\\'

In [10]:
#prepare cross_validation
num_conc = pd.read_csv(data_file, sep=',').iloc[:,0]

for fold in k_folds:
    split_path = case_path + 'split_'+ str(fold[0])+'_'+str(fold[1])+ '\\'

    os.makedirs(split_path, exist_ok = True)
    y_trn, y_30 = train_test_split(np.array(num_conc), test_size=0.3, random_state=fold[0])
    y_vld, y_tst = train_test_split(y_30, test_size=0.333, random_state=fold[1])

    pd.DataFrame(y_trn, columns = ['pattern_number']).to_csv(split_path+'Y_trn.csv', sep=',',index=False)
    pd.DataFrame(y_vld, columns = ['pattern_number']).to_csv(split_path+'Y_vld.csv', sep=',',index=False)
    pd.DataFrame(y_tst, columns = ['pattern_number']).to_csv(split_path+'Y_tst.csv', sep=',',index=False)

In [10]:
model = OneDCNN().float()

loss_function = torch.nn.MSELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
l2_lambda = 0.001

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134],[139,15],[287,403],[32,29]] #cross-validation folds
k_folds = [[287,403],[32,29]]
a = ['sample_number',target]

for fold in k_folds:
    
    split_path = case_path + 'split_'+ str(fold[0])+'_'+str(fold[1])+ '\\'
    
    #Data import and normalization
    training_data =  IR_wine_dataset(targets_num[target],split_path+'Y_trn.csv',data_file,label_file)

    train_dataloader = DataLoader(training_data, batch_size=1213, shuffle=True)
    mean_f, std_f = mean_std(train_dataloader)

    training_data =  IR_wine_dataset(targets_num[target],split_path+'Y_trn.csv',data_file,label_file, transform= transforms.Compose([Normalize(mean_f, std_f)]))
    validation_data =  IR_wine_dataset(targets_num[target],split_path+'Y_vld.csv',data_file,label_file, transform= transforms.Compose([Normalize(mean_f, std_f)]))
    test_data =  IR_wine_dataset(targets_num[target],split_path+'Y_tst.csv',data_file,label_file, transform= transforms.Compose([Normalize(mean_f, std_f)]))

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
    

    for init_number in range(0,5): #Это я делаю множественную инициализацию весов сети

        init_path = split_path + str(init_number)+'\\'
        os.makedirs(init_path, exist_ok=True)

        # Обучение
        # критерием остановки является ошибка на валидационном наборе - останавливаемся,
        #если в течение 100 эпох (test_stop) ошибка на валидационном наборе (val_loss) не падала

        test_stop = 100 #stopping criterion
        max_val_loss = 10000.0

        wandb.init(project = case_name)

        split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])
        init_name = '_' + str(init_number)
        model_name = '_CNN'
        wandb.run.name = split_name + init_name + model_name
        wandb.run.save()

        for epoch_step in range(0, 10000, test_stop):
            
            if epoch_step!=0:
                checkpoint = torch.load(init_path + 'model'+model_name+'.pth')
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                last_best_epoch = checkpoint['epoch']
                loss = checkpoint['loss']
                model.train()
                
                if last_best_epoch + test_stop > ep:
                    for ep in range(epoch_step, epoch_step+test_stop):
                        for _, data in enumerate(train_dataloader, 0): # get bacth
                            num, inputs, labels = data # parse batch
                            optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                            outputs = model(inputs) # get outputs
                            loss = loss_function(outputs, labels) # calculate loss
                            #Regularization
                            l2_norm = sum(p.pow(2.0).sum()
                                      for p in model.parameters())

                            loss = loss + l2_lambda * l2_norm
                            loss.backward() # calculate gradients
                            optimizer.step() # performs a single optimization step (parameter update).

                        dl = 0
                        val_loss = 0.0
                        for num, specs, labels in validation_dataloader: 
                            val_loss += loss_function(model(specs),labels) + l2_lambda * l2_norm
                            dl+=1
                        val_loss = val_loss/dl
                        
                        if val_loss.item() <= max_val_loss:
                            torch.save({'epoch': ep,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'loss': loss}, init_path + 'model'+model_name+'.pth')
                            max_val_loss = val_loss.item()
                        wandb.log({"trn_loss": loss, "vld_loss": val_loss})
                else: continue    
      
            if epoch_step==0:
                model.apply(reset_weights)

                for ep in range(epoch_step, test_stop):
                    for _, data in enumerate(train_dataloader, 0): # get bacth
                        num, inputs, labels = data # parse batch
                        optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                        outputs = model(inputs) # get outputs
                        loss = loss_function(outputs, labels) # calculate loss
                        #Regularization
                        l2_norm = sum(p.pow(2.0).sum()
                                      for p in model.parameters())

                        loss = loss + l2_lambda * l2_norm
                        
                        loss.backward() # calculate gradients
                        optimizer.step() # performs a single optimization step (parameter update).

                    dl = 0
                    val_loss = 0.0
                    for num, specs, labels in validation_dataloader:
                        val_loss += loss_function(model(specs),labels) + l2_lambda * l2_norm
                        dl+=1
                    val_loss = val_loss/dl

                    if val_loss.item() <= max_val_loss:
                        torch.save({'epoch': ep,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'loss': loss}, init_path + 'model'+model_name+'.pth')
                        max_val_loss = val_loss.item()
                    wandb.log({"trn_loss": loss, "vld_loss": val_loss})

        wandb.log({"epoch": ep})
        wandb.finish()

        write_predictions(target, model, model_name, init_path, train_dataloader, out_num, dset = 'trn')
        write_predictions(target, model, model_name, init_path, validation_dataloader, out_num, dset ='vld')
        write_predictions(target, model, model_name, init_path, test_dataloader, out_num, dset ='tst')


wandb: Currently logged in as: oe_sarmanova (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
C:\Users\helga\anaconda3\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Reset trainable parameters of layer = Conv1d(1, 4, kernel_size=(9,), stride=(1,))
Reset trainable parameters of layer = Conv1d(4, 8, kernel_size=(9,), stride=(1,))
Reset trainable parameters of layer = Linear(in_features=3688, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=1, bias=True)


trn_loss,0.24776
vld_loss,0.50279
_step,900
_runtime,3190
_timestamp,1701289529
epoch,899


trn_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,▁


...

wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Conv1d(1, 4, kernel_size=(9,), stride=(1,))
Reset trainable parameters of layer = Conv1d(4, 8, kernel_size=(9,), stride=(1,))
Reset trainable parameters of layer = Linear(in_features=3688, out_features=32, bias=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=1, bias=True)


trn_loss,0.19909
vld_loss,0.79728
_step,500
_runtime,1321
_timestamp,1701304850
epoch,499


trn_loss,█▅▅▄▅▅▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,█▄▄▄▄▄▄▄▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁
